# 🏀 バスケットボール選手判別チャレンジ - EDA

このノートブックでは、データの理解を深めるための探索的データ分析を行います。

## 目次
1. データ読み込み
2. データ前処理検討

In [ ]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import matplotlib_fontja

import warnings
warnings.filterwarnings('ignore')

# original
import sys
sys.path.append(r"..")
import utils
from utils.data import sep, show_df, glob_walk, set_seed

# 日本語フォント設定
# plt.rcParams['font.family'] = ['Noto Sans CJK JP', 'Yu Gothic', 'Hiragino Sans', 'MS Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

print("ライブラリの読み込み完了")


In [ ]:
%matplotlib widget
import matplotlib
print(matplotlib.get_backend())

In [ ]:
# パス設定
DATA_DIR = Path("../data/raw")
IMAGE_DIR = Path(DATA_DIR / "images")

In [ ]:
# メタデータ読み込み
train_meta = pd.read_csv(DATA_DIR / "train_meta.csv")
test_meta = pd.read_csv(DATA_DIR / "test_meta.csv")
test_top_meta = pd.read_csv(DATA_DIR / "test_top_meta.csv")
sample_submission = pd.read_csv(DATA_DIR / "sample_submission.csv")

print("データ読み込み完了")
print(f"train_meta: {train_meta.shape}")
print(f"test_meta: {test_meta.shape}")
print(f"test_top_meta: {test_top_meta.shape}")
print(f"sample_submission: {sample_submission.shape}")

In [ ]:
def build_image_presence_table(img_dir: Path) -> pd.DataFrame:
    """
    画像ファイル名: {quarter}__{angle}__{session}__{frame}.jpg
      例: Q2-019__top__00__44.jpg

    戻り値:
      quarter, session, frame, is_top, is_side, top_path, side_path
    """
    rows = []
    bad = []

    # jpg/JPG/jpeg/JPEG も拾う
    exts = {".jpg", ".jpeg"}
    files = [p for p in img_dir.rglob("*") if p.is_file() and p.suffix.lower() in exts]

    for p in files:
        stem = p.stem
        parts = stem.split("__")
        if len(parts) != 4:
            bad.append(p.name)
            continue

        quarter, angle, session_str, frame_str = parts
        if angle not in ("top", "side"):
            bad.append(p.name)
            continue

        try:
            session = int(session_str)  # "00" -> 0
            frame = int(frame_str)      # "44" -> 44
        except ValueError:
            bad.append(p.name)
            continue

        rows.append(
            {
                "quarter": quarter,
                "angle": angle,
                "session": session,
                "frame": frame,
                "path": str(p),  # 絶対パス/相対パスどちらでもOK
            }
        )

    df_files = pd.DataFrame(rows)
    if len(df_files) == 0:
        raise RuntimeError(f"No valid image files found under: {img_dir}")

    # (quarter, session, frame) ごとに top/side のパスを横持ちにする
    df_pivot = (
        df_files.pivot_table(
            index=["quarter", "session", "frame"],
            columns="angle",
            values="path",
            aggfunc="first",
        )
        .rename(columns={"top": "top_path", "side": "side_path"})
        .reset_index()
    )

    # 片方しか存在しない場合に備えて列を補完
    if "top_path" not in df_pivot.columns:
        df_pivot["top_path"] = pd.NA
    if "side_path" not in df_pivot.columns:
        df_pivot["side_path"] = pd.NA

    df_pivot["is_top"] = df_pivot["top_path"].notna().astype("int8")
    df_pivot["is_side"] = df_pivot["side_path"].notna().astype("int8")

    # 見やすい順に並べる
    df_pivot = df_pivot[["quarter", "session", "frame", "is_top", "is_side", "top_path", "side_path"]]
    df_pivot = df_pivot.sort_values(["quarter", "session", "frame"]).reset_index(drop=True)

    # 変なファイル名が混じってたら、必要なら確認用に出す
    if bad:
        print(f"[WARN] Skipped {len(bad)} files due to unexpected filename format. Example:", bad[:5])

    return df_pivot

In [ ]:
img_meta = build_image_presence_table(IMAGE_DIR)

# どれくらい「両方ある/片方だけ」があるかチェック
print(img_meta.groupby(["is_top", "is_side"]).size().rename("n_frames"))
display(img_meta.head())

In [ ]:
train = pd.read_csv(DATA_DIR / "train_meta.csv")

# dtype事故防止（特に session/frame は int に）
train["session"] = train["session"].astype(int)
train["frame"] = train["frame"].astype(int)

train = train.merge(
    img_meta,
    on=["quarter", "session", "frame"],
    how="left",
    validate="m:1",
)

# このbbox行が指している画像パス（trainの angle に応じて選ぶ）
train["image_path"] = np.where(train["angle"] == "top", train["top_path"], train["side_path"])

# 反対側の画角があるならそれ（補助特徴として使うとき便利）
train["paired_path"] = np.where(train["angle"] == "top", train["side_path"], train["top_path"])

# 画像が見つからない行がないかチェック
missing = train["image_path"].isna().mean()
print(f"train image_path missing rate: {missing:.6f}")

display(train.head())

In [ ]:
test = pd.read_csv(DATA_DIR / "test_meta.csv")
test["session"] = test["session"].astype(int)
test["frame"] = test["frame"].astype(int)

test = test.merge(img_meta, on=["quarter", "session", "frame"], how="left", validate="m:1")

# test_metaは side 固定なら image_path は side_path でOK
test["image_path"] = test["side_path"]

# top が存在するなら補助として
test["paired_top_path"] = test["top_path"]

print("test missing side image rate:", test["image_path"].isna().mean())
display(test.head())

In [ ]:
test_top = pd.read_csv(DATA_DIR / "test_top_meta.csv")
test_top["session"] = test_top["session"].astype(int)
test_top["frame"] = test_top["frame"].astype(int)

test_top = test_top.merge(img_meta, on=["quarter", "session", "frame"], how="left", validate="m:1")
test_top["image_path"] = test_top["top_path"]

print("test_top missing top image rate:", test_top["image_path"].isna().mean())
display(test_top.head())

# ホモグラフィー検討

In [ ]:
# 1フレームあたりbboxが何個あるか（test）
cnt = test.groupby(["quarter","session","frame"]).size()
print(cnt.describe())
print(cnt.value_counts().head(20))

In [ ]:
%matplotlib widget
import cv2, numpy as np, matplotlib.pyplot as plt
import time

def pick_points_blocking(img_bgr, n=4, title="Click points", timeout=180):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(figsize=(12,6))
    ax.imshow(img_rgb)
    ax.set_title(f"{title} | click {n} points: TL, TR, BR, BL")
    pts = []

    def onclick(event):
        if event.inaxes != ax: return
        if event.xdata is None or event.ydata is None: return
        pts.append([float(event.xdata), float(event.ydata)])
        ax.plot(event.xdata, event.ydata, "rx")
        fig.canvas.draw()

    cid = fig.canvas.mpl_connect("button_press_event", onclick)
    plt.show()

    t0 = time.time()
    while len(pts) < n:
        plt.pause(0.1)
        if time.time() - t0 > timeout:
            fig.canvas.mpl_disconnect(cid)
            plt.close(fig)
            raise TimeoutError(f"Timeout: got {len(pts)}/{n} points")

    fig.canvas.mpl_disconnect(cid)
    plt.close(fig)
    return np.asarray(pts[:n], dtype=np.float32)

# W, H = 960, 540
# dst = np.array([[0,0], [W,0], [W,H], [0,H]], dtype=np.float32)

# img_top  = cv2.imread(str(IMAGE_DIR / "Q1-000__top__00__01.jpg"))
# img_side = cv2.imread(str(IMAGE_DIR / "Q1-000__side__00__01.jpg"))

# src_top  = pick_points_blocking(img_top, 4, "TOP: TL, TR, BR, BL")
# H_top    = cv2.getPerspectiveTransform(src_top, dst)

# src_side = pick_points_blocking(img_side, 4, "SIDE: TL, TR, BR, BL")
# H_side   = cv2.getPerspectiveTransform(src_side, dst)

# warp_top  = cv2.warpPerspective(img_top,  H_top,  (W,H))
# warp_side = cv2.warpPerspective(img_side, H_side, (W,H))

# fig, ax = plt.subplots(figsize=(12,5))
# ax.imshow(cv2.cvtColor(warp_top, cv2.COLOR_BGR2RGB))
# ax.set_title("warped top")
# ax.axis("off")
# plt.show()

# fig, ax = plt.subplots(figsize=(12,5))
# ax.imshow(cv2.cvtColor(warp_side, cv2.COLOR_BGR2RGB))
# ax.set_title("warped side")
# ax.axis("off")
# plt.show()

In [ ]:
%matplotlib widget
import cv2, numpy as np, matplotlib.pyplot as plt

img_top = cv2.imread(str(IMAGE_DIR / "Q1-000__top__00__01.jpg"))
img_rgb = cv2.cvtColor(img_top, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(figsize=(10,5))
im = ax.imshow(img_rgb)
ax.set_title("TOP: move mouse (x,y shown). Zoom/pan OK. No clicking needed.")
ax.axis("off")

txt = ax.text(10, 20, "", color="yellow", fontsize=12,
              bbox=dict(facecolor="black", alpha=0.5, pad=3))

def on_move(event):
    if event.inaxes != ax or event.xdata is None or event.ydata is None:
        return
    txt.set_text(f"x={event.xdata:.1f}, y={event.ydata:.1f}")
    fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

In [ ]:
import numpy as np, cv2

W, H = 960, 540
dst = np.array([[0,0],[W,0],[W,H],[0,H]], dtype=np.float32)

src_top = np.array([
    [  420.4,  218.6],  # TL
    [ 3399.9,  275.4],  # BL
    [  3379.1, 1901.7],  # BR
    [ 366.6, 1825.3],  # TR
], dtype=np.float32)

H_top = cv2.getPerspectiveTransform(src_top, dst)

In [ ]:
warp_top = cv2.warpPerspective(img_top, H_top, (W, H))
plt.figure(figsize=(10,5))
plt.imshow(cv2.cvtColor(warp_top, cv2.COLOR_BGR2RGB))
plt.title("warped TOP")
plt.axis("off")
plt.show()

In [ ]:
img_side = cv2.imread(str(IMAGE_DIR / "Q1-000__side__00__01.jpg"))
img_rgb = cv2.cvtColor(img_side, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(figsize=(10,5))
im = ax.imshow(img_rgb)
ax.set_title("TOP: move mouse (x,y shown). Zoom/pan OK. No clicking needed.")
ax.axis("off")

txt = ax.text(10, 20, "", color="yellow", fontsize=12,
              bbox=dict(facecolor="black", alpha=0.5, pad=3))

def on_move(event):
    if event.inaxes != ax or event.xdata is None or event.ydata is None:
        return
    txt.set_text(f"x={event.xdata:.1f}, y={event.ydata:.1f}")
    fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

In [ ]:
W, H = 960, 540
dst = np.array([[0,0],[W,0],[W,H],[0,H]], dtype=np.float32)

src_side = np.array([
    [ 978.27,  954.74],   # TL
    [2841.03,  952.84],   # TR  ←ここ
    [3437.96, 1324.81],   # BR
    [ 387.09, 1335.23],   # BL  ←ここ
], dtype=np.float32)

H_side = cv2.getPerspectiveTransform(src_side, dst)
warp_side = cv2.warpPerspective(img_side, H_side, (W, H))  # dsizeは(width,height)

In [ ]:
warp_side = cv2.warpPerspective(img_side, H_side, (W, H))
plt.figure(figsize=(10,5))
plt.imshow(cv2.cvtColor(warp_side, cv2.COLOR_BGR2RGB))
plt.title("warped TOP")
plt.axis("off")
plt.show()

In [ ]:
def add_court_xy(df, H_top, H_side, W, H):
    df = df.copy()

    # px は常に bbox の横中心
    px = df["x"].values + df["w"].values * 0.5

    # py は angle で切り替え
    py = np.where(
        df["angle"].values == "top",
        df["y"].values + df["h"].values * 0.5,  # top: center
        df["y"].values + df["h"].values         # side: bottom
    )

    pts = np.stack([px, py], axis=1).astype(np.float32).reshape(-1, 1, 2)
    out = np.zeros((len(df), 2), dtype=np.float32)

    is_top = (df["angle"].values == "top")
    if is_top.any():
        out[is_top] = cv2.perspectiveTransform(pts[is_top], H_top).reshape(-1, 2)
    if (~is_top).any():
        out[~is_top] = cv2.perspectiveTransform(pts[~is_top], H_side).reshape(-1, 2)

    # 正規化
    df["court_x"] = out[:, 0] / W
    df["court_y"] = out[:, 1] / H

    return df

In [ ]:
add_court_xy(df=train, H_top=H_top, H_side=H_side, W=960, H=540)

# アフィン変換を諦める

In [ ]:
# ============================
# Formation features (one cell)
# - auto img_w/img_h per angle
# - add features for train/test/test_top
# - keep original row order (important for submission)
# ============================

import numpy as np
import pandas as pd
import cv2

# ---------- 0) utils: get (w,h) per angle ----------
def get_img_size_from_path(path: str):
    img = cv2.imread(str(path))
    if img is None:
        raise FileNotFoundError(f"image not found or unreadable: {path}")
    h, w = img.shape[:2]
    return w, h

def get_angle_img_sizes(df, angle_col="angle", path_col="image_path"):
    """Return dict: angle -> (img_w, img_h) using the first available image_path for each angle."""
    sizes = {}
    for ang, g in df.groupby(angle_col):
        p = g[path_col].dropna().iloc[0]
        sizes[ang] = get_img_size_from_path(p)
    return sizes

# ---------- 1) anchor point per bbox ----------
def bbox_anchor_xy(df, img_w, img_h):
    """
    top : bbox center
    side: bottom-center (foot proxy)
    return normalized (N,2) in [0,1] (may slightly exceed)
    """
    x = df["x"].to_numpy(np.float32)
    y = df["y"].to_numpy(np.float32)
    w = df["w"].to_numpy(np.float32)
    h = df["h"].to_numpy(np.float32)

    cx = x + 0.5 * w
    cy = np.where(
        df["angle"].to_numpy() == "top",
        y + 0.5 * h,   # top: center
        y + h          # side: bottom
    )

    nx = cx / float(img_w)
    ny = cy / float(img_h)
    return np.stack([nx, ny], axis=1).astype(np.float32)

# ---------- 2) normalize for translation/scale/rotation (PCA) ----------
def normalize_points_rigid(P, eps=1e-6):
    """
    P: (N,2)  normalized image coords
    returns Q: (N,2) after removing translation + scale + rotation (approx)
    """
    c = P.mean(axis=0, keepdims=True)
    X = P - c

    s = np.sqrt((X**2).sum(axis=1).mean()) + eps
    X = X / s

    C = (X.T @ X) / X.shape[0]
    eigvals, eigvecs = np.linalg.eigh(C)  # ascending
    v1 = eigvecs[:, 1]  # principal
    v2 = eigvecs[:, 0]
    R = np.stack([v1, v2], axis=1)        # (2,2)

    Q = X @ R

    # fix sign ambiguity (avoid 180° flips)
    if Q[:, 0].max() < -Q[:, 0].min():
        Q *= -1.0

    return Q.astype(np.float32), float(s)

# ---------- 3) per-frame formation features ----------
def add_formation_features(
    df: pd.DataFrame,
    img_sizes: dict,  # angle -> (w,h)
    key_cols=("quarter", "session", "frame"),
):
    df = df.copy()
    df["__row_id"] = np.arange(len(df), dtype=np.int64)

    out_chunks = []
    for keys, g in df.groupby(list(key_cols), sort=False):
        # g は元dfの index を保持しているので、ローカル位置を作る
        g = g.copy()
        g["__pos"] = np.arange(len(g), dtype=np.int32)  # 0..len(g)-1

        P = np.zeros((len(g), 2), dtype=np.float32)

        for ang, gg in g.groupby("angle", sort=False):
            img_w, img_h = img_sizes[ang]
            P_ang = bbox_anchor_xy(gg, img_w, img_h)

            # ローカル位置で代入（ここが修正点）
            pos = gg["__pos"].to_numpy()
            P[pos] = P_ang

        Qn, shape_scale = normalize_points_rigid(P)

        center = P.mean(axis=0)
        dist_center = np.sqrt(((P - center) ** 2).sum(axis=1))

        rank_x = pd.Series(P[:, 0]).rank(method="first").to_numpy(np.int32) - 1
        rank_y = pd.Series(P[:, 1]).rank(method="first").to_numpy(np.int32) - 1

        if len(P) >= 2:
            D = np.sqrt(((P[:, None, :] - P[None, :, :]) ** 2).sum(axis=2)).astype(np.float32)
            np.fill_diagonal(D, np.nan)          # ← 自分自身は除外
            nn_dist = np.nanmin(D, axis=1)       # 最近傍（自分除外）
            mean_dist = np.nanmean(D, axis=1)    # 平均距離（自分除外）
        else:
            nn_dist = np.full((len(P),), np.nan, dtype=np.float32)
            mean_dist = np.full((len(P),), np.nan, dtype=np.float32)

        gg_out = g.copy()
        gg_out["fx"] = Qn[:, 0]
        gg_out["fy"] = Qn[:, 1]
        gg_out["dist_center"] = dist_center.astype(np.float32)
        gg_out["rank_x"] = rank_x.astype(np.int16)
        gg_out["rank_y"] = rank_y.astype(np.int16)
        gg_out["nn_dist"] = nn_dist.astype(np.float32)
        gg_out["mean_dist"] = mean_dist.astype(np.float32)
        gg_out["shape_scale"] = np.float32(shape_scale)

        gg_out["bbox_area"] = (gg_out["w"].to_numpy(np.float32) * gg_out["h"].to_numpy(np.float32))
        gg_out["bbox_aspect"] = (gg_out["w"].to_numpy(np.float32) / (gg_out["h"].to_numpy(np.float32) + 1e-6))

        # 後処理
        gg_out = gg_out.drop(columns="__pos")

        out_chunks.append(gg_out)

    out = pd.concat(out_chunks, axis=0)
    out = out.sort_values("__row_id").drop(columns="__row_id").reset_index(drop=True)
    return out

In [ ]:
# ---------- 4) apply to train/test/test_top (order preserved) ----------
# img size dict is built from the union to cover both angles
# (train contains both; test is side-only; test_top is top-only)
union_df = pd.concat(
    [
        train[["angle", "image_path"]].dropna(),
        test[["angle", "image_path"]].dropna(),
        test_top[["angle", "image_path"]].dropna(),
    ],
    axis=0,
    ignore_index=True
).drop_duplicates()
show_df(union_df,3,True)

# get_angle_img_sizes
img_sizes = get_angle_img_sizes(union_df, angle_col="angle", path_col="image_path")
print("img_sizes:", img_sizes)

train_f = add_formation_features(train, img_sizes=img_sizes)
test_f  = add_formation_features(test,  img_sizes=img_sizes)
test_top_f = add_formation_features(test_top, img_sizes=img_sizes)
sep("train_f"); show_df(train_f,3,True)
sep("test_f"); show_df(test_f)
sep("test_top_f"); show_df(test_top_f)


# submission order guarantee:
# test_f row order == original test row order (same as sample_submission.csv)
# quick check: index alignment
assert len(test_f) == len(test), "row count changed!"

In [ ]:
train_f.describe()

# もう一度トライ

In [ ]:
import cv2
import numpy as np

def order_points_tltrbrbl(pts):
    """pts: (4,2) -> TL,TR,BR,BL"""
    pts = np.asarray(pts, dtype=np.float32)
    s = pts.sum(axis=1)          # x+y
    d = np.diff(pts, axis=1)[:,0]# x-y
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(d)]
    bl = pts[np.argmax(d)]
    return np.stack([tl, tr, br, bl], axis=0).astype(np.float32)

def detect_court_corners_by_green(img_bgr, debug=False):
    """
    Return 4 corners (TL,TR,BR,BL) of the largest green-ish region using minAreaRect.
    Works best for top view.
    """
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    # 緑のレンジ（まずはこれ。環境により調整）
    lower = np.array([35, 40, 40], dtype=np.uint8)
    upper = np.array([95, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)

    # ノイズ除去＆穴埋め
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)

    # 最大輪郭
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        raise RuntimeError("No contour found. Try adjusting HSV thresholds.")

    cnt = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    if area < 10_000:
        raise RuntimeError(f"Contour too small: {area}. Threshold likely wrong.")

    # 回転矩形の4隅
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)  # (4,2) float
    corners = order_points_tltrbrbl(box)

    if debug:
        dbg = img_bgr.copy()
        cv2.drawContours(dbg, [corners.astype(int)], -1, (0,0,255), 3)
        for i,(x,y) in enumerate(corners.astype(int)):
            cv2.putText(dbg, ["TL","TR","BR","BL"][i], (x+5,y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
        return corners, mask, dbg

    return corners

In [ ]:
W, H = 960, 540
dst = np.array([[0,0],[W,0],[W,H],[0,H]], dtype=np.float32)

img_top = cv2.imread(str(IMAGE_DIR / "Q1-000__top__00__01.jpg"))
corners_top, mask_top, dbg_top = detect_court_corners_by_green(img_top, debug=True)

H_top = cv2.getPerspectiveTransform(corners_top, dst)
warp_top = cv2.warpPerspective(img_top, H_top, (W, H))

import matplotlib.pyplot as plt
plt.figure(figsize=(12,4)); plt.imshow(cv2.cvtColor(dbg_top, cv2.COLOR_BGR2RGB)); plt.title("detected corners"); plt.axis("off")
plt.figure(figsize=(12,4)); plt.imshow(mask_top, cmap="gray"); plt.title("green mask"); plt.axis("off")
plt.figure(figsize=(12,4)); plt.imshow(cv2.cvtColor(warp_top, cv2.COLOR_BGR2RGB)); plt.title("warped top"); plt.axis("off")
plt.show()

In [ ]:
def detect_side_with_roi(img_bgr, y0_ratio=0.25, y1_ratio=0.92, debug=False):
    h, w = img_bgr.shape[:2]
    y0 = int(h * y0_ratio)
    y1 = int(h * y1_ratio)
    roi = img_bgr[y0:y1, :].copy()

    out = detect_court_corners_by_green(roi, debug=debug)
    if debug:
        corners, mask, dbg = out
    else:
        corners = out

    # ROI座標 -> 元画像座標に戻す
    corners[:,1] += y0

    if debug:
        # dbgはROI上なので、元画像に重ねたければ別途描く
        return corners, mask
    return corners

In [ ]:
# ============================
# Batch validation for court-corner detection (green mask)
# - samples N frames per angle from your image folder
# - runs corner detection
# - reports success rate + saves debug mosaics
# ============================

from pathlib import Path
import random
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt

IMG_DIR = Path("../data/raw/images")  # ←適宜

# --------- from previous message ----------
def order_points_tltrbrbl(pts):
    pts = np.asarray(pts, dtype=np.float32)
    s = pts.sum(axis=1)
    d = np.diff(pts, axis=1)[:, 0]
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(d)]
    bl = pts[np.argmax(d)]
    return np.stack([tl, tr, br, bl], axis=0).astype(np.float32)

def detect_court_corners_by_green(img_bgr, debug=False,
                                  lower=np.array([35, 40, 40], np.uint8),
                                  upper=np.array([95,255,255], np.uint8),
                                  ksize=11, open_iter=1, close_iter=2,
                                  min_area=10_000):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ksize, ksize))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=open_iter)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=close_iter)

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        raise RuntimeError("no contour")

    cnt = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    if area < min_area:
        raise RuntimeError(f"too small area={area}")

    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    corners = order_points_tltrbrbl(box)

    if debug:
        dbg = img_bgr.copy()
        cv2.drawContours(dbg, [corners.astype(int)], -1, (0,0,255), 3)
        for i,(x,y) in enumerate(corners.astype(int)):
            cv2.putText(dbg, ["TL","TR","BR","BL"][i], (x+5,y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
        return corners, mask, dbg, area

    return corners, mask, area

def detect_side_with_roi(img_bgr, y0_ratio=0.25, y1_ratio=0.92, **kwargs):
    h, w = img_bgr.shape[:2]
    y0 = int(h * y0_ratio)
    y1 = int(h * y1_ratio)
    roi = img_bgr[y0:y1, :].copy()
    corners, mask, area = detect_court_corners_by_green(roi, debug=False, **kwargs)
    corners[:, 1] += y0
    return corners, mask, area

# --------- file sampling ----------
pat = re.compile(r"^(Q\d-\d{3})__(top|side)__\d+__\d+\.(jpg|jpeg)$", re.IGNORECASE)

def list_images(img_dir: Path):
    files = []
    for p in img_dir.rglob("*"):
        if not p.is_file(): 
            continue
        if p.suffix.lower() not in [".jpg", ".jpeg"]:
            continue
        m = pat.match(p.name)
        if not m:
            continue
        quarter, angle = m.group(1), m.group(2).lower()
        files.append((quarter, angle, p))
    return files

all_files = list_images(IMG_DIR)
print("found:", len(all_files), "images")

def sample_files(files, angle, n=20, seed=42):
    cand = [x for x in files if x[1] == angle]
    random.Random(seed).shuffle(cand)
    return cand[:min(n, len(cand))]

# --------- run batch ----------
def run_batch(angle="top", n=20, seed=42,
              use_roi_for_side=True,
              hsv_lower=(35,40,40), hsv_upper=(95,255,255),
              ksize=11, open_iter=1, close_iter=2, min_area=10_000):
    samp = sample_files(all_files, angle, n=n, seed=seed)
    results = []
    for quarter, ang, p in samp:
        img = cv2.imread(str(p))
        try:
            if ang == "side" and use_roi_for_side:
                corners, mask, area = detect_side_with_roi(
                    img,
                    lower=np.array(hsv_lower, np.uint8),
                    upper=np.array(hsv_upper, np.uint8),
                    ksize=ksize, open_iter=open_iter, close_iter=close_iter,
                    min_area=min_area
                )
                # make debug image (draw corners on full image)
                dbg = img.copy()
                cv2.drawContours(dbg, [corners.astype(int)], -1, (0,0,255), 3)
            else:
                corners, mask, dbg, area = detect_court_corners_by_green(
                    img, debug=True,
                    lower=np.array(hsv_lower, np.uint8),
                    upper=np.array(hsv_upper, np.uint8),
                    ksize=ksize, open_iter=open_iter, close_iter=close_iter,
                    min_area=min_area
                )
            ok = True
            err = ""
        except Exception as e:
            ok = False
            err = str(e)
            corners = None
            mask = None
            dbg = img

        results.append({
            "quarter": quarter,
            "angle": ang,
            "path": str(p),
            "ok": ok,
            "error": err,
            "dbg": dbg,
            "mask": mask,
        })

    df_res = (np.array([(r["quarter"], r["path"], r["ok"], r["error"]) for r in results],
                      dtype=object))
    ok_rate = sum(r["ok"] for r in results) / len(results) if results else 0
    print(f"[{angle}] n={len(results)} ok_rate={ok_rate:.2%}")
    if results:
        bad = [r for r in results if not r["ok"]]
        if bad:
            print("  failures (first 5):")
            for r in bad[:5]:
                print("   -", r["path"], ":", r["error"])

    return results

# change n as you like
top_results  = run_batch(angle="top",  n=16, seed=1, use_roi_for_side=False)
# side_results = run_batch(angle="side", n=16, seed=1, use_roi_for_side=True)

# --------- show mosaics ----------
def show_mosaic(results, title, cols=4, use_mask=False, max_items=16):
    items = results[:max_items]
    rows = int(np.ceil(len(items) / cols))
    plt.figure(figsize=(4*cols, 3*rows))
    for i, r in enumerate(items):
        img = r["mask"] if use_mask else r["dbg"]
        if img is None:
            img = np.zeros((100,100,3), dtype=np.uint8)
        plt.subplot(rows, cols, i+1)
        if use_mask:
            plt.imshow(img, cmap="gray")
        else:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        name = Path(r["path"]).name
        plt.title(("OK " if r["ok"] else "NG ") + name, fontsize=8)
        plt.axis("off")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

show_mosaic(top_results,  "TOP debug (corners)", cols=4, use_mask=False)
show_mosaic(top_results,  "TOP mask", cols=4, use_mask=True)

# show_mosaic(side_results, "SIDE debug (corners) [ROI]", cols=4, use_mask=False)
# show_mosaic(side_results, "SIDE mask [ROI]", cols=4, use_mask=True)

In [ ]:
import cv2, numpy as np, matplotlib.pyplot as plt

# ---------- utils ----------
def keep_largest_cc(mask):
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num <= 1:
        return mask
    idx = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    out = np.zeros_like(mask)
    out[labels == idx] = 255
    return out

def fill_holes(mask):
    h, w = mask.shape[:2]
    inv = cv2.bitwise_not(mask)
    ff = inv.copy()
    tmp = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(ff, tmp, (0,0), 255)
    ff_inv = cv2.bitwise_not(ff)
    return cv2.bitwise_or(mask, ff_inv)

def green_mask_side_roi(img_bgr, y0, y1, x0, x1,
                        lower=(35,70,40), upper=(95,255,255),
                        close_ksize=31, close_iter=2):
    roi = img_bgr[y0:y1, x0:x1].copy()
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array(lower,np.uint8), np.array(upper,np.uint8))

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_ksize, close_ksize))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=close_iter)

    mask = keep_largest_cc(mask)
    mask = fill_holes(mask)
    return mask

def corners_from_mask_scanlines_quantile(mask, x0, y0, band=10, pad=3, min_white=40,
                                         q_left=0.01, q_right=0.99):
    m = (mask > 0)
    rows_all = np.where(m.any(axis=1))[0]
    if len(rows_all) == 0:
        raise RuntimeError("mask has no white pixels")

    top_r0 = int(np.clip(rows_all[0] + pad, 0, m.shape[0]-1))
    bot_r0 = int(np.clip(rows_all[-1] - pad, 0, m.shape[0]-1))

    top_rows = range(top_r0, min(top_r0 + band, m.shape[0]))
    bot_rows = range(max(0, bot_r0 - band + 1), bot_r0 + 1)

    def collect_lr(rows):
        lefts, rights, ys = [], [], []
        for r in rows:
            cols = np.where(m[r])[0]
            if len(cols) >= min_white:
                lefts.append(cols[0]); rights.append(cols[-1]); ys.append(r)
        if len(lefts) == 0:
            raise RuntimeError("empty band (min_white too strict or ROI wrong)")
        return np.array(lefts), np.array(rights), np.array(ys)

    top_lefts, top_rights, top_ys = collect_lr(top_rows)
    bot_lefts, bot_rights, bot_ys = collect_lr(bot_rows)

    tl = np.array([np.quantile(top_lefts,  q_left),  np.median(top_ys)], np.float32)
    tr = np.array([np.quantile(top_rights, q_right), np.median(top_ys)], np.float32)
    bl = np.array([np.quantile(bot_lefts,  q_left),  np.median(bot_ys)], np.float32)
    br = np.array([np.quantile(bot_rights, q_right), np.median(bot_ys)], np.float32)

    # ROI -> full image
    tl += (x0, y0); tr += (x0, y0); br += (x0, y0); bl += (x0, y0)

    return np.stack([tl, tr, br, bl], axis=0).astype(np.float32)

In [ ]:
import numpy as np

def corners_from_mask_4sides(
    mask, x0, y0,
    band=10, pad=3,
    q_low=0.01, q_high=0.99,
    min_white_ratio=0.01, min_white_floor=5
):
    """
    mask: ROI内 0/255
    return TL,TR,BR,BL (full-image coords)
    """
    m = (mask > 0)
    H, W = m.shape
    rows = np.where(m.any(axis=1))[0]
    cols = np.where(m.any(axis=0))[0]
    if len(rows) == 0 or len(cols) == 0:
        raise RuntimeError("mask empty")

    # ROI幅に応じてmin_whiteを自動化
    min_white = max(min_white_floor, int(W * min_white_ratio))

    # --- top/bottom y を band で推定 ---
    top_r0 = int(np.clip(rows[0] + pad, 0, H-1))
    bot_r0 = int(np.clip(rows[-1] - pad, 0, H-1))

    top_rows = range(top_r0, min(top_r0 + band, H))
    bot_rows = range(max(0, bot_r0 - band + 1), bot_r0 + 1)

    def pick_y(rows_iter):
        ys = []
        widths = []
        for r in rows_iter:
            cc = np.where(m[r])[0]
            if len(cc) >= min_white:
                ys.append(r)
                widths.append(len(cc))
        if len(ys) == 0:
            # fallback: 1本でも白がある行を使う
            for r in rows_iter:
                cc = np.where(m[r])[0]
                if len(cc) > 0:
                    ys.append(r); widths.append(len(cc))
        if len(ys) == 0:
            raise RuntimeError("cannot pick y")
        # 幅が大きい行ほど安定なので、幅で重み付けした分位点
        ys = np.array(ys); widths = np.array(widths)
        # ここは単純に中央値でOK
        return float(np.median(ys))

    y_top = pick_y(top_rows)
    y_bot = pick_y(bot_rows)

    # --- left/right x を band で推定（列方向） ---
    left_c0 = int(np.clip(cols[0] + pad, 0, W-1))
    right_c0 = int(np.clip(cols[-1] - pad, 0, W-1))

    left_cols = range(left_c0, min(left_c0 + band, W))
    right_cols = range(max(0, right_c0 - band + 1), right_c0 + 1)

    def pick_x(cols_iter):
        xs = []
        heights = []
        for c in cols_iter:
            rr = np.where(m[:, c])[0]
            if len(rr) >= min_white:  # ここは同じmin_whiteでOK
                xs.append(c)
                heights.append(len(rr))
        if len(xs) == 0:
            for c in cols_iter:
                rr = np.where(m[:, c])[0]
                if len(rr) > 0:
                    xs.append(c); heights.append(len(rr))
        if len(xs) == 0:
            raise RuntimeError("cannot pick x")
        return float(np.median(np.array(xs)))

    x_left = pick_x(left_cols)
    x_right = pick_x(right_cols)

    # --- corners in ROI coords ---
    tl = np.array([x_left,  y_top], np.float32)
    tr = np.array([x_right, y_top], np.float32)
    br = np.array([x_right, y_bot], np.float32)
    bl = np.array([x_left,  y_bot], np.float32)

    # ROI -> full image
    tl += (x0, y0); tr += (x0, y0); br += (x0, y0); bl += (x0, y0)

    return np.stack([tl,tr,br,bl], axis=0).astype(np.float32)

In [ ]:
import cv2, numpy as np, matplotlib.pyplot as plt

# img_side = cv2.imread(str(IMAGE_DIR / "Q1-000__side__00__01.jpg"))
# ---------- single test ----------
img_side = cv2.imread(str(IMAGE_DIR / "Q2-007__side__00__38.jpg"))
H_img, W_img = img_side.shape[:2]

y0, y1 = 850, 1400
x0 = int(W_img * 0.05)
x1 = int(W_img * 0.95)

mask = green_mask_side_roi(img_side, y0=y0, y1=y1, x0=x0, x1=x1,
                           lower=(35,70,40), upper=(95,255,255),
                           close_ksize=31, close_iter=2)

corners = corners_from_mask_4sides(mask, x0=x0, y0=y0, band=10, pad=3)

Wout, Hout = 960, 540
dst = np.array([[0,0],[Wout,0],[Wout,Hout],[0,Hout]], np.float32)
H_side = cv2.getPerspectiveTransform(corners, dst)
warp_side = cv2.warpPerspective(img_side, H_side, (Wout, Hout))

dbg = img_side.copy()
cv2.polylines(dbg, [corners.astype(int)], True, (0,0,255), 3)

plt.figure(figsize=(12,4)); plt.imshow(cv2.cvtColor(dbg, cv2.COLOR_BGR2RGB)); plt.title("side dbg corners (4-sides)"); plt.axis("off")
plt.figure(figsize=(12,2)); plt.imshow(mask, cmap="gray"); plt.title("mask(cleaned)"); plt.axis("off")
plt.figure(figsize=(12,4)); plt.imshow(cv2.cvtColor(warp_side, cv2.COLOR_BGR2RGB)); plt.title("warped side"); plt.axis("off")
plt.show()

In [ ]:
from pathlib import Path
import re, random
import numpy as np
import cv2
import matplotlib.pyplot as plt

IMG_DIR = Path("../data/raw/images")  # 適宜
Wout, Hout = 960, 540
dst = np.array([[0,0],[Wout,0],[Wout,Hout],[0,Hout]], np.float32)

# ---------- helpers ----------
pat = re.compile(r"^(Q\d-\d{3})__(side)__\d+__\d+\.(jpg|jpeg)$", re.IGNORECASE)

def list_side_images(img_dir: Path):
    out = []
    for p in img_dir.rglob("*"):
        if not p.is_file(): 
            continue
        if p.suffix.lower() not in [".jpg", ".jpeg"]:
            continue
        m = pat.match(p.name)
        if not m:
            continue
        quarter = m.group(1)
        out.append((quarter, p))
    return out

def keep_largest_cc(mask):
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num <= 1:
        return mask
    idx = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    out = np.zeros_like(mask)
    out[labels == idx] = 255
    return out

def fill_holes(mask):
    h, w = mask.shape[:2]
    inv = cv2.bitwise_not(mask)
    ff = inv.copy()
    tmp = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(ff, tmp, (0,0), 255)
    ff_inv = cv2.bitwise_not(ff)
    return cv2.bitwise_or(mask, ff_inv)

# def green_mask_side_roi(img_bgr, y0=850, y1=1400, lower=(35,70,40), upper=(95,255,255),
def green_mask_side_roi(img_bgr, y0=850, y1=1400, lower=(35,70,40), upper=(95,255,255),
                        close_ksize=31, close_iter=2):
    H, W = img_bgr.shape[:2]
    y0 = max(0, min(H-1, int(y0)))
    y1 = max(0, min(H,   int(y1)))
    roi = img_bgr[y0:y1, :].copy()

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array(lower,np.uint8), np.array(upper,np.uint8))

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_ksize, close_ksize))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=close_iter)

    mask = keep_largest_cc(mask)
    mask = fill_holes(mask)
    return roi, mask

def corners_from_mask_scanlines_quantile(mask, x0=0, y0=0, band=10, pad=3, min_white=80,
                                         q_left=0.01, q_right=0.99):
    m = (mask > 0)
    rows_all = np.where(m.any(axis=1))[0]
    if len(rows_all) == 0:
        raise RuntimeError("mask has no white pixels")

    top_r0 = int(np.clip(rows_all[0] + pad, 0, m.shape[0]-1))
    bot_r0 = int(np.clip(rows_all[-1] - pad, 0, m.shape[0]-1))

    top_rows = range(top_r0, min(top_r0 + band, m.shape[0]))
    bot_rows = range(max(0, bot_r0 - band + 1), bot_r0 + 1)

    def collect_lr(rows):
        lefts, rights, ys = [], [], []
        for r in rows:
            cols = np.where(m[r])[0]
            if len(cols) >= min_white:
                lefts.append(cols[0]); rights.append(cols[-1]); ys.append(r)
        if len(lefts) == 0:
            raise RuntimeError("empty band (min_white too strict or ROI wrong)")
        return np.array(lefts), np.array(rights), np.array(ys)

    top_lefts, top_rights, top_ys = collect_lr(top_rows)
    bot_lefts, bot_rights, bot_ys = collect_lr(bot_rows)

    tl = np.array([np.quantile(top_lefts,  q_left),  np.median(top_ys)], np.float32)
    tr = np.array([np.quantile(top_rights, q_right), np.median(top_ys)], np.float32)
    bl = np.array([np.quantile(bot_lefts,  q_left),  np.median(bot_ys)], np.float32)
    br = np.array([np.quantile(bot_rights, q_right), np.median(bot_ys)], np.float32)

    tl += (x0, y0); tr += (x0, y0); br += (x0, y0); bl += (x0, y0)
    return np.stack([tl,tr,br,bl], axis=0).astype(np.float32)

def warp_quality_score(warp_bgr):
    """超簡易：HSVで緑の割合を見る（コートが映っているか）"""
    hsv = cv2.cvtColor(warp_bgr, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array([35,40,40],np.uint8), np.array([95,255,255],np.uint8))
    green_ratio = (mask > 0).mean()
    return float(green_ratio)

# ---------- batch runner ----------
def batch_validate_side(
    n=50,
    seed=0,
    # y0=850, y1=1400,
    y0=700, y1=1550,
    lower=(35,70,40), upper=(95,255,255),
    close_ksize=31, close_iter=2,
    band_mode="ratio", band_ratio=0.02, band_min=8,
    min_white=80,
    q_left=0.01, q_right=0.99,
    green_ratio_min=0.15, green_ratio_max=0.95,  # 成功判定のゆるい範囲
):
    files = list_side_images(IMG_DIR)
    random.Random(seed).shuffle(files)
    files = files[:min(n, len(files))]

    results = []
    for quarter, p in files:
        img = cv2.imread(str(p))
        try:
            roi, mask = green_mask_side_roi(
                img, y0=y0, y1=y1, lower=lower, upper=upper,
                close_ksize=close_ksize, close_iter=close_iter
            )

            H_img, W_img = img.shape[:2]
            roi_h = (y1 - y0)
            if band_mode == "ratio":
                band = max(band_min, int(roi_h * band_ratio))
            else:
                band = band_min  # 固定

            corners = corners_from_mask_scanlines_quantile(
                mask, x0=0, y0=y0, band=band, pad=3, min_white=min_white,
                q_left=q_left, q_right=q_right
            )
            Hm = cv2.getPerspectiveTransform(corners, dst)
            warp = cv2.warpPerspective(img, Hm, (Wout, Hout))

            gr = warp_quality_score(warp)
            ok = (green_ratio_min <= gr <= green_ratio_max)

            dbg = img.copy()
            cv2.polylines(dbg, [corners.astype(int)], True, (0,0,255), 3)

            results.append({
                "path": str(p),
                "quarter": quarter,
                "ok": ok,
                "green_ratio": gr,
                "dbg": dbg,
                "mask": mask,
                "warp": warp,
                "band": band,
            })
        except Exception as e:
            results.append({
                "path": str(p),
                "quarter": quarter,
                "ok": False,
                "green_ratio": None,
                "dbg": img,
                "mask": None,
                "warp": None,
                "band": None,
                "error": str(e),
            })

    ok_rate = sum(r["ok"] for r in results) / len(results) if results else 0
    print(f"[SIDE] n={len(results)} ok_rate={ok_rate:.2%}")
    print("green_ratio (ok only):",
          np.mean([r["green_ratio"] for r in results if r["ok"]]) if any(r["ok"] for r in results) else None)

    # 失敗理由のサマリ
    errs = [r.get("error","") for r in results if not r["ok"] and r.get("error")]
    if errs:
        from collections import Counter
        c = Counter([e.split("\n")[0] for e in errs])
        print("top errors:", c.most_common(5))

    return results

def show_mosaic(results, key="dbg", title="", cols=4, max_items=12):
    items = results[:max_items]
    rows = int(np.ceil(len(items)/cols))
    plt.figure(figsize=(4*cols, 3*rows))
    for i,r in enumerate(items):
        img = r.get(key)
        if img is None:
            img = np.zeros((100,100,3), np.uint8)
        plt.subplot(rows, cols, i+1)
        if key == "mask":
            plt.imshow(img, cmap="gray")
        else:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        name = Path(r["path"]).name
        gr = r.get("green_ratio")
        lab = ("OK " if r["ok"] else "NG ") + name
        if gr is not None:
            lab += f"\n gr={gr:.2f} band={r.get('band')}"
        plt.title(lab, fontsize=8)
        plt.axis("off")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# ---------- run ----------
results = batch_validate_side(n=40, seed=1, band_mode="ratio", band_ratio=0.02, band_min=8)

ok_list = [r for r in results if r["ok"]]
ng_list = [r for r in results if not r["ok"]]

show_mosaic(ok_list, key="dbg",  title="SIDE OK (dbg corners)", cols=4, max_items=8)
show_mosaic(ng_list, key="dbg",  title="SIDE NG (dbg corners)", cols=4, max_items=8)
show_mosaic(ng_list, key="mask", title="SIDE NG masks", cols=4, max_items=8)
show_mosaic(ng_list, key="warp", title="SIDE NG warps", cols=4, max_items=8)